# Convolutional Layer

In [10]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import cv2

In [11]:
from convolution2d import Convolution as cv

### CL Class

In [63]:
path = '/Users/rajeshr/Desktop/Group_22/selected/'

images = []
color_images = []
label = []
for i in os.listdir(path):
    image = cv2.imread(path+i, cv2.IMREAD_GRAYSCALE)
    color_image = cv2.imread(path+i, cv2.COLOR_BGR2RGB)
    try:
        images.append(cv2.resize(image, (224, 224)))
        color_images.append(cv2.resize(color_image, (224, 224)))
        label.append(i)
    except:
        pass
    
images = np.array(images)
label = np.array(label)
color_images = np.array(color_images)

In [64]:
color_images.shape

(3, 224, 224, 3)

In [18]:
class layer():
    
    def __init__(self):
        pass
    
    def kaiming(self, n, size):
        mean, sd = 0, np.sqrt(2/n)
        weights = np.random.normal(mean, sd, size=size)
        return weights
    
    def ReLUActivation(self, x):
        out = []
        for i in x:
            for j in i:
                if j>=0:
                    out.append(j)
                else:
                    out.append(0)
        return np.array(out).reshape(x.shape)
    
    def ConVlayer(self, image, K, size, n):
        output = []
        ReLUout = []
        filters = []
        for i in range(K):
            filtr = self.kaiming(n, size)
            filters.append(filtr)
            res = cv.convolve(cv, inpt=image, filtr=filtr, stride = 1, padding = 0)
            output.append(res)
            ReLUout.append(self.ReLUActivation(res))
            
        self.output = np.array(output)
        self.ReLUout = np.array(ReLUout)
        self.filters = np.array(filters)
        
    def ConVDepthlayer(self, images, K, size, n):
        output = []
        ReLUout = []
        filters = []
        for i in range(K):
            filtr = self.kaiming(n, size)
            filters.append(filtr)
            res = cv.convwithDepth(cv, inpt=images, filtr=filtr, stride = 1, padding = 0)
            output.append(res)
            ReLUout.append(self.ReLUActivation(res))
        
        self.output = np.array(output)
        self.ReLUout = np.array(ReLUout)
        self.filters = np.array(filters)
        
    def poolingLayer(self, featureMaps, size, stride):
        m, n = featureMaps[0].shape
        fm, fn = size
        result = []
        
        poolout = []
        for featureMap in featureMaps:
            for i in range(0, m, stride):
                for j in range(0, n, stride):
                    if featureMap[i:i+fm, j:j+fn].shape == size:
                        result.append(max(featureMap[i:i+fm, j:j+fn].flatten()))
                        
            Outm, Outn = int(((m-fm)/stride)+1), int(((n-fn)/stride)+1)
            poolout.append(np.array(result).reshape((Outm, Outn)))
            result = []
        
        self.poolout = np.array(poolout)

In [138]:
#GRAY SCALE
image = images[2]    

layer1 = layer()
layer1.ConVlayer(image, 32, (3, 3), 9)

layer2 = layer()
layer2.ConVDepthlayer(layer1.ReLUout, 64, (3, 3), 9)

In [139]:
label

array(['ketch.jpg', 'chandelier.jpg', 'bonsai.jpg'], dtype='<U14')

In [67]:
layer1_filters = layer1.filters.reshape(layer1.filters.shape[0], -1)
layer2_filters = layer2.filters.reshape(layer2.filters.shape[0], -1)

np.savetxt('/Users/rajeshr/Desktop/layer1filter.txt', layer1_filters, delimiter=',')
np.savetxt('/Users/rajeshr/Desktop/layer2filter.txt', layer2_filters, delimiter=',')

In [167]:
k = 1
for filtr in layer1.filters:
    cv2.imwrite('/Users/rajeshr/Desktop/l1/filter'+str(k)+'.jpg', filtr*255)
    k += 1

k = 1
for filtr in layer2.filters:
    cv2.imwrite('/Users/rajeshr/Desktop/l2/filter'+str(k)+'.jpg', filtr*255)
    k += 1